In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the libraries
import pandas as pd
import json

In [3]:
# read the USDM JSON for the LZT study
in_file = "data/usdm.json"
try:
    with open(in_file, 'r') as f:
        usdm = json.load(f)
except FileNotFoundError:
    print('The file {} was not found...'.format(in_file))

In [4]:
# Return the STUDYID
studyId = usdm['study']['versions'][0]['studyIdentifiers'][0]['text']

In [5]:
# Create the required study dictionary lists
elements = usdm['study']['versions'][0]['studyDesigns'][0]['elements']
cells = usdm['study']['versions'][0]['studyDesigns'][0]['studyCells']
arms = usdm['study']['versions'][0]['studyDesigns'][0]['arms']
epochs = usdm['study']['versions'][0]['studyDesigns'][0]['epochs']
encounters = usdm['study']['versions'][0]['studyDesigns'][0]['encounters']

In [16]:
# Create the elements dataframe
elementsDF = pd.DataFrame(elements, columns=['id','name','label','description','transitionStartRule','transitionEndRule'])
startRules = pd.json_normalize(elementsDF.transitionStartRule)
startRules.columns = [f'startRule_{col}' for col in startRules.columns]
endRules = pd.json_normalize(elementsDF.transitionEndRule)
endRules.columns = [f'endRule_{col}' for col in endRules.columns]
elementsDF = pd.concat([elementsDF,startRules,endRules], axis=1)
elementsDF = elementsDF.drop(columns=['transitionStartRule','transitionEndRule','startRule_id','startRule_name','startRule_extensionAttributes','startRule_label','startRule_description',
                             'startRule_instanceType','endRule_id','endRule_name','endRule_extensionAttributes','endRule_label','endRule_description','endRule_instanceType'])
elementsDF.rename(columns={'id':'elements.id', 'name':'elements.name', 'label':'elements.label', 'description':'elements.description', 'startRule_text':'elements.startRule_text',
                          'endRule_text':'elements.endRule_text'}, inplace=True)
elementsDF

,elements.id,elements.name,elements.label,elements.description,elements.startRule_text,elements.endRule_text
0,StudyElement_1,EL1,Screening,Screening Element,Informed consent,Completion of all screening activities and no ...
1,StudyElement_2,EL2,Placebo,Placebo TTS (adhesive patches),Administration of first dose,NaN
2,StudyElement_7,EL7,Follow up,Follow Up Element,End of last scheduled visit on study (includin...,Completion of all specified followup activitie...
3,StudyElement_3,EL3,Low,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN
4,StudyElement_4,EL4,High - Start,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Randomized,NaN
5,StudyElement_5,EL5,High - Middle,"Xanomeline TTS (adhesive patches) 50 cm2, 54 m...",Administration of first dose (from patches sup...,NaN
6,StudyElement_6,EL6,High - End,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose (from patches sup...,NaN


In [17]:
# Create the epochs dataframe
epochsDF = pd.DataFrame(epochs, columns=['id','name','label','description','type','previousId','nextId'])

types = pd.json_normalize(epochsDF.type)
types.columns = [f'type_{col}' for col in types.columns]
epochsDF = pd.concat([epochsDF,types], axis=1)
epochsDF = epochsDF.drop(columns=['type', 'type_id', 'type_extensionAttributes', 'type_codeSystem', 'type_codeSystemVersion', 'type_instanceType'])
epochsDF.rename(columns={'id':'epochs.id','name':'epochs.name','label':'epochs.label','description':'epochs.description','previousId':'epochs.previousId',
                        'nextId':'epochs.nextId','type_code':'epochs.type_code','type_decode':'epochs.type_decode'}, inplace=True)
epochsDF.insert(1, 'epochs.taetord', range(1, 1+len(epochsDF)))
epochsDF

,epochs.id,epochs.taetord,epochs.name,epochs.label,epochs.description,epochs.previousId,epochs.nextId,epochs.type_code,epochs.type_decode
0,StudyEpoch_1,1,Screening,Screening,Screening Epoch,None,StudyEpoch_2,C202487,Screening Epoch
1,StudyEpoch_2,2,Treatment 1,Treatment One,Treatment Epoch,StudyEpoch_1,StudyEpoch_3,C101526,Treatment Epoch
2,StudyEpoch_3,3,Treatment 2,Treatment Two,Treatment Epoch,StudyEpoch_2,StudyEpoch_4,C101526,Treatment Epoch
3,StudyEpoch_4,4,Treatment 3,Treatment Three,Treatment Epoch,StudyEpoch_3,StudyEpoch_5,C101526,Treatment Epoch
4,StudyEpoch_5,5,Follow-Up,Follow Up,Follow-up Epoch,StudyEpoch_4,None,C202578,Follow-Up Epoch


In [18]:
# Create the arms dataframe
armsDF = pd.DataFrame(arms, columns=['id', 'name', 'label', 'description', 'type', 'dataOriginType'])
types = pd.json_normalize(armsDF.type)
types.columns = [f'type_{col}' for col in types.columns]
origins = pd.json_normalize(armsDF.dataOriginType)
origins.columns = [f'dataOriginType_{col}' for col in origins.columns]
armsDF = pd.concat([armsDF,types,origins], axis=1)
armsDF = armsDF.drop(columns=['type', 'type_id', 'type_extensionAttributes', 'type_codeSystem', 'type_codeSystemVersion', 'dataOriginType',
                             'type_instanceType', 'dataOriginType_id', 'dataOriginType_extensionAttributes', 'dataOriginType_codeSystem', 
                             'dataOriginType_codeSystemVersion', 'dataOriginType_instanceType'])
armsDF.rename(columns={'id':'arms.id','name':'arms.name','label':'arms.label','description':'arms.description','type_code':'arms.type_code',
                      'type_decode':'arms.type_decode','dataOriginType_code':'arms.dataOriginType_code', 'dataOriginType_decode':'arms.dataOriginType_decode'}, inplace=True)
armsDF

,arms.id,arms.name,arms.label,arms.description,arms.type_code,arms.type_decode,arms.dataOriginType_code,arms.dataOriginType_decode
0,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,C188866,Data Generated Within Study
1,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,C188866,Data Generated Within Study
2,StudyArm_3,Xanomeline High Dose,Xanomeline High Dose,Active Substance,C174267,Active Comparator Arm,C188866,Data Generated Within Study


In [22]:
# Create the cells dataframe
cellsDF = pd.DataFrame(cells, columns = ['id','armId','epochId','elementIds'])
cellsDF['elementId'] = cellsDF['elementIds'].apply(lambda x: ''.join(map(str, x)))
cellsDF = cellsDF.drop(columns=['elementIds'])
cellsDF.rename(columns={'id':'cells.id','armId':'cells.armId','epochId':'cells.epochId','elementId':'cells.elementId'}, inplace=True)
cellsDF

,cells.id,cells.armId,cells.epochId,cells.elementId
0,StudyCell_1,StudyArm_1,StudyEpoch_1,StudyElement_1
1,StudyCell_2,StudyArm_1,StudyEpoch_2,StudyElement_2
2,StudyCell_3,StudyArm_1,StudyEpoch_3,StudyElement_2
3,StudyCell_4,StudyArm_1,StudyEpoch_4,StudyElement_2
4,StudyCell_5,StudyArm_1,StudyEpoch_5,StudyElement_7
5,StudyCell_6,StudyArm_2,StudyEpoch_1,StudyElement_1
6,StudyCell_7,StudyArm_2,StudyEpoch_2,StudyElement_3
7,StudyCell_8,StudyArm_2,StudyEpoch_3,StudyElement_3
8,StudyCell_9,StudyArm_2,StudyEpoch_4,StudyElement_3
9,StudyCell_10,StudyArm_2,StudyEpoch_5,StudyElement_7


In [25]:
# Merge the cells and arms dataframes
ta_df = pd.merge(cellsDF, armsDF, how='left', left_on='cells.armId', right_on='arms.id')
ta_df = pd.merge(ta_df, epochsDF, how='left', left_on='cells.epochId', right_on='epochs.id')
ta_df = pd.merge(ta_df, elementsDF, how='left', left_on='cells.elementId', right_on='elements.id')
ta_df

,cells.id,cells.armId,cells.epochId,cells.elementId,arms.id,arms.name,arms.label,arms.description,arms.type_code,arms.type_decode,...,epochs.previousId,epochs.nextId,epochs.type_code,epochs.type_decode,elements.id,elements.name,elements.label,elements.description,elements.startRule_text,elements.endRule_text
0,StudyCell_1,StudyArm_1,StudyEpoch_1,StudyElement_1,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,...,None,StudyEpoch_2,C202487,Screening Epoch,StudyElement_1,EL1,Screening,Screening Element,Informed consent,Completion of all screening activities and no ...
1,StudyCell_2,StudyArm_1,StudyEpoch_2,StudyElement_2,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,...,StudyEpoch_1,StudyEpoch_3,C101526,Treatment Epoch,StudyElement_2,EL2,Placebo,Placebo TTS (adhesive patches),Administration of first dose,NaN
2,StudyCell_3,StudyArm_1,StudyEpoch_3,StudyElement_2,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,...,StudyEpoch_2,StudyEpoch_4,C101526,Treatment Epoch,StudyElement_2,EL2,Placebo,Placebo TTS (adhesive patches),Administration of first dose,NaN
3,StudyCell_4,StudyArm_1,StudyEpoch_4,StudyElement_2,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,...,StudyEpoch_3,StudyEpoch_5,C101526,Treatment Epoch,StudyElement_2,EL2,Placebo,Placebo TTS (adhesive patches),Administration of first dose,NaN
4,StudyCell_5,StudyArm_1,StudyEpoch_5,StudyElement_7,StudyArm_1,Placebo,Placebo,Placebo,C174268,Placebo Control Arm,...,StudyEpoch_4,None,C202578,Follow-Up Epoch,StudyElement_7,EL7,Follow up,Follow Up Element,End of last scheduled visit on study (includin...,Completion of all specified followup activitie...
5,StudyCell_6,StudyArm_2,StudyEpoch_1,StudyElement_1,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,...,None,StudyEpoch_2,C202487,Screening Epoch,StudyElement_1,EL1,Screening,Screening Element,Informed consent,Completion of all screening activities and no ...
6,StudyCell_7,StudyArm_2,StudyEpoch_2,StudyElement_3,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,...,StudyEpoch_1,StudyEpoch_3,C101526,Treatment Epoch,StudyElement_3,EL3,Low,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN
7,StudyCell_8,StudyArm_2,StudyEpoch_3,StudyElement_3,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,...,StudyEpoch_2,StudyEpoch_4,C101526,Treatment Epoch,StudyElement_3,EL3,Low,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN
8,StudyCell_9,StudyArm_2,StudyEpoch_4,StudyElement_3,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,...,StudyEpoch_3,StudyEpoch_5,C101526,Treatment Epoch,StudyElement_3,EL3,Low,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN
9,StudyCell_10,StudyArm_2,StudyEpoch_5,StudyElement_7,StudyArm_2,Xanomeline Low Dose,Xanomeline Low Dose,Active Substance,C174267,Active Comparator Arm,...,StudyEpoch_4,None,C202578,Follow-Up Epoch,StudyElement_7,EL7,Follow up,Follow Up Element,End of last scheduled visit on study (includin...,Completion of all specified followup activitie...


## Create the final TA Trial Design Domain ##

***TODO:** Need an example for calculating TATRANS, TABRANCH*

In [27]:
# Create the final TA trial design domain
ta = pd.DataFrame(ta_df, columns=['arms.name','arms.description','epochs.taetord','elements.name','elements.description','epochs.name'])
ta['STUDYID'] = studyId
ta['DOMAIN'] = 'TA'
ta['TABRANCH'] = None
ta['TATRANS'] = None
ta.rename(columns={'arms.name':'ARMCD','arms.description':'ARM','epochs.taetord':'TAETORD','elements.name':'ETCD','elements.description':'ELEMENT','epochs.name':'EPOCH'}, inplace=True)
ta = ta[['STUDYID','DOMAIN','ARMCD','ARM','TAETORD','ETCD','ELEMENT','TABRANCH','TATRANS','EPOCH']]
ta

,STUDYID,DOMAIN,ARMCD,ARM,TAETORD,ETCD,ELEMENT,TABRANCH,TATRANS,EPOCH
0,H2Q-MC-LZZT,TA,Placebo,Placebo,1,EL1,Screening Element,None,None,Screening
1,H2Q-MC-LZZT,TA,Placebo,Placebo,2,EL2,Placebo TTS (adhesive patches),None,None,Treatment 1
2,H2Q-MC-LZZT,TA,Placebo,Placebo,3,EL2,Placebo TTS (adhesive patches),None,None,Treatment 2
3,H2Q-MC-LZZT,TA,Placebo,Placebo,4,EL2,Placebo TTS (adhesive patches),None,None,Treatment 3
4,H2Q-MC-LZZT,TA,Placebo,Placebo,5,EL7,Follow Up Element,None,None,Follow-Up
5,H2Q-MC-LZZT,TA,Xanomeline Low Dose,Active Substance,1,EL1,Screening Element,None,None,Screening
6,H2Q-MC-LZZT,TA,Xanomeline Low Dose,Active Substance,2,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",None,None,Treatment 1
7,H2Q-MC-LZZT,TA,Xanomeline Low Dose,Active Substance,3,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",None,None,Treatment 2
8,H2Q-MC-LZZT,TA,Xanomeline Low Dose,Active Substance,4,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",None,None,Treatment 3
9,H2Q-MC-LZZT,TA,Xanomeline Low Dose,Active Substance,5,EL7,Follow Up Element,None,None,Follow-Up


## Create the final TE Trial Design Domain

***TODO:** Create logic to calculate TEDUR
Select scheduleInstances that relate to start of the associated StudyEpoch associated with the corresponding
study Element via StudyCell. Select the scheduleInstance associated with the start of the next studyEpoch. Use
Timing.values of all related timings that specify the period inbetween for calculation of the total element duration.*



In [28]:
# Create the final TE Trial Design Domain
te = pd.DataFrame(ta_df,columns=['elements.name','elements.description','elements.startRule_text','elements.endRule_text'])
te['STUDYID'] = studyId
te['DOMAIN'] = 'TE'
te['TEDUR'] = None
te.rename(columns={'elements.name':'ETCD','elements.description':'ELEMENT','elements.startRule_text':'TESTRL','elements.endRule_text':'TEENRL'}, inplace=True)
te = te[['STUDYID','DOMAIN','ETCD','ELEMENT','TESTRL','TEENRL','TEDUR']]
te

,STUDYID,DOMAIN,ETCD,ELEMENT,TESTRL,TEENRL,TEDUR
0,H2Q-MC-LZZT,TE,EL1,Screening Element,Informed consent,Completion of all screening activities and no ...,None
1,H2Q-MC-LZZT,TE,EL2,Placebo TTS (adhesive patches),Administration of first dose,NaN,None
2,H2Q-MC-LZZT,TE,EL2,Placebo TTS (adhesive patches),Administration of first dose,NaN,None
3,H2Q-MC-LZZT,TE,EL2,Placebo TTS (adhesive patches),Administration of first dose,NaN,None
4,H2Q-MC-LZZT,TE,EL7,Follow Up Element,End of last scheduled visit on study (includin...,Completion of all specified followup activitie...,None
5,H2Q-MC-LZZT,TE,EL1,Screening Element,Informed consent,Completion of all screening activities and no ...,None
6,H2Q-MC-LZZT,TE,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN,None
7,H2Q-MC-LZZT,TE,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN,None
8,H2Q-MC-LZZT,TE,EL3,"Xanomeline TTS (adhesive patches) 50 cm2, 54 mg",Administration of first dose,NaN,None
9,H2Q-MC-LZZT,TE,EL7,Follow Up Element,End of last scheduled visit on study (includin...,Completion of all specified followup activitie...,None


## Create the final TV Trial Design Domain ##

*Because the trial visits are the same for all arms, the ARMCD should be left blank*

In [29]:
# create the encounters dataframe
encountersDF = pd.DataFrame(encounters, columns=['id','name','label','description','type','previousId','nextId',
                                                 'scheduledAtId','transitionStartRule','transitionEndRule'])
types = pd.json_normalize(encountersDF.type)
types.columns = [f'type_{col}' for col in types.columns]
startRules = pd.json_normalize(encountersDF.transitionStartRule)
startRules.columns = [f'startRule_{col}' for col in startRules.columns]
endRules = pd.json_normalize(encountersDF.transitionEndRule)
endRules.columns = [f'endRule_{col}' for col in endRules.columns]
encountersDF = pd.concat([encountersDF,types,startRules,endRules], axis=1)
encountersDF = encountersDF.drop(columns=['type','transitionStartRule','transitionEndRule','startRule_description',
                                          'startRule_extensionAttributes','startRule_instanceType','type_extensionAttributes',
                                         'startRule_name','startRule_label','type_codeSystem','type_codeSystemVersion',
                                         'type_instanceType','endRule_extensionAttributes','endRule_name','endRule_label',
                                         'endRule_description','endRule_instanceType'])
encountersDF.rename(columns={'id':'encounter.id','name':'encounter.name','label':'encounter.label','description':'encounter.description',
                             'previousId':'encounter.previousId','nextId':'encounter.nextId','scheduledAtId':'encounter.scheduledAtId',
                             'type_id':'encounter.type_id','type_code':'encounter.type_code','type_decode':'encounter.type_decode',
                             'startRule_id':'encounter.startRule_id','startRule_text':'encounter.startRule_text','endRule_id':'encounter.endRule_id',
                             'endRule_text':'encounter.endRule_text'}, inplace=True)

#encountersDF.head()

*Create a timings dataframe for the retrieval of VISITDY variable*

In [30]:
timings = usdm['study']['versions'][0]['studyDesigns'][0]['scheduleTimelines'][0]['timings']
timingsDF = pd.DataFrame(timings, columns=['id','name','label','description','value'])
timingsDF.rename(columns={'id':'timing.id','name':'timing.name','label':'timing.label','description':'timing.description','value':'timing.value'}, inplace=True)
#timingsDF.head()

In [31]:
# Merge encounters and timings dataframes to include the VISITDY (timings.timing.value)
tv = pd.merge(encountersDF, timingsDF, how='left', left_on='encounter.scheduledAtId', right_on='timing.id')
tv = pd.DataFrame(tv, columns=['encounter.name','timing.value','encounter.startRule_text','encounter.endRule_text'])
tv['STUDYID'] = studyId
tv['DOMAIN'] = 'TV'
tv.insert(1, 'encounter.visitnum', range(1, 1+len(encountersDF)))
tv.rename(columns={'encounter.name':'VISIT','encounter.visitnum':'VISITNUM','timing.value':'VISITDY','encounter.startRule_text':'TVSTRL','encounter.endRule_text':'TVENRL'}, inplace=True)
tv = tv[['STUDYID','DOMAIN','VISITNUM','VISIT','VISITDY','TVSTRL','TVENRL']]
tv

,STUDYID,DOMAIN,VISITNUM,VISIT,VISITDY,TVSTRL,TVENRL
0,H2Q-MC-LZZT,TV,1,E1,NaN,Subject identifier,completion of screening activities
1,H2Q-MC-LZZT,TV,2,E2,P2D,NaN,subject leaves clinic after connection of ambu...
2,H2Q-MC-LZZT,TV,3,E3,NaN,subject has connection of ambulatory ECG machi...,Radomized
3,H2Q-MC-LZZT,TV,4,E4,P2W,NaN,NaN
4,H2Q-MC-LZZT,TV,5,E5,P4W,NaN,NaN
5,H2Q-MC-LZZT,TV,6,E7,P6W,NaN,NaN
6,H2Q-MC-LZZT,TV,7,E8,P8W,NaN,NaN
7,H2Q-MC-LZZT,TV,8,E9,P12W,NaN,NaN
8,H2Q-MC-LZZT,TV,9,E10,P16W,NaN,NaN
9,H2Q-MC-LZZT,TV,10,E11,P20W,NaN,NaN


*NOTE: There are issues with the USDM for this study.  The values for the encounter start and end rule text should be completed for each encounter.*

**encounter[i]['transitionStartRule']['text']**

**encounter[i]['transitionEndRule']['text']**

## Create the final TI Trial Design Domain

In [39]:
criteria = usdm['study']['versions'][0]['studyDesigns'][0]['eligibilityCriteria']
criteriaDF = pd.DataFrame(criteria, columns=['id','name','label','description','category','identifier'])
categories = pd.json_normalize(criteriaDF.category)
categories.columns = [f'category_{col}' for col in categories.columns]
criteriaDF = pd.concat([criteriaDF, categories], axis=1)
ti = pd.DataFrame(criteriaDF, columns=['name','label','category_decode'])
ti.rename(columns={'name':'IETESTCD','label':'IETEST','category_decode':'IECAT'}, inplace=True)
ti['STUDYID'] = studyId
ti['DOMAIN'] = 'TI'
ti = ti[['STUDYID','DOMAIN','IETESTCD','IETEST','IECAT']]
ti

,STUDYID,DOMAIN,IETESTCD,IETEST,IECAT
0,H2Q-MC-LZZT,TI,IN01,Age greater than 50,Inclusion Criteria
1,H2Q-MC-LZZT,TI,IN02,Diagnosis of Alzheimer's,Inclusion Criteria
2,H2Q-MC-LZZT,TI,IN03,MMSE Score,Inclusion Criteria
3,H2Q-MC-LZZT,TI,IN04,Hachinski Ischemic Score,Inclusion Criteria
4,H2Q-MC-LZZT,TI,IN05,CNS imaging comptaible with Alzheimer's,Inclusion Criteria
5,H2Q-MC-LZZT,TI,IN06,Informed consent criteria,Inclusion Criteria
6,H2Q-MC-LZZT,TI,IN07,Geographic proximity criteria,Inclusion Criteria
7,H2Q-MC-LZZT,TI,IN08,Reliable caregiver criteria,Inclusion Criteria
8,H2Q-MC-LZZT,TI,EX01,Previous study criteria,Exclusion Criteria
9,H2Q-MC-LZZT,TI,EX02,Other Alzheimer's therapy criteria,Exclusion Criteria
